In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import scipy.stats
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import json

plt.style.use('../peptidome.mplstyle')

import sys
sys.path.append('..')
from lib import *

In [ ]:
class memoized(object):
    """Decorator. Caches a function's return value each time it is called.
    If called later with the same arguments, the cached value is returned
    (not reevaluated).
    """
    def __init__(self, func):
        self.func = func
        self.cache = {}

    def __call__(self, *args):
        # if args is not Hashable we can't cache
        # easier to ask for forgiveness than permission
        try:
            if args in self.cache:
                return self.cache[args]
            else:
                value = self.func(*args)
                self.cache[args] = value
                return value
        except TypeError:
            return self.func(*args)

In [ ]:
from subprocess import check_output
from io import StringIO
@memoized
def normalize_taxid(taxid, rank='species'):
    'return species level taxon id'
    if not taxid:
        return ''
    out = check_output('efetch -db taxonomy -id "{taxid}"  -format native -mode xml -json'.format(taxid=taxid), shell=True, text=True)
    buffer = StringIO(out)
    results = json.load(buffer)
    if results['TaxaSet']['Taxon']['Rank'] == 'species':
        print(taxid, 'is species')
        return taxid
    try:
        print(taxid, len(results['TaxaSet']['Taxon']['LineageEx']['Taxon']))
        ids = [level['TaxId'] for level in results['TaxaSet']['Taxon']['LineageEx']['Taxon'] if level['Rank'] == rank]
    except (KeyError, TypeError):
        return ''
    if len(ids) == 0:
        return ''
    return ids[0] 

In [ ]:
df = load_unirefproteome_as_df_path(datadir + 'human-viruses-uniref90.fasta')
df['length'] = df['Sequence'].str.len()
df.sort_values('length', ascending=False, inplace=True)
df = df[~df['header'].str.contains('(Fragment)')]

In [ ]:
df['TaxID_species'] = df['TaxID'].apply(normalize_taxid)

In [ ]:
def parse_protein_name(header):
    m = re.search('(?<=\s).+(?=\sn\=)', header)
    if m:
         return m.group(0)
    return ''

In [ ]:
df['Protein name'] = df.apply(lambda row: parse_protein_name(row['header']), axis=1)

In [ ]:
df['TaxID_species'].value_counts().head(20)

In [ ]:
df = df[~(df['TaxID_species']=='')]

In [ ]:
len(set(df['TaxID'].unique())), len(set(df['TaxID_species'].unique()))

In [ ]:
plt.hist(np.log10(df['TaxID_species'].value_counts()))

In [ ]:
def counter_to_series(counter):
    count_df = pd.DataFrame.from_dict(counter, orient='index', columns=['count'])
    count_series = count_df.T.squeeze()
    return count_series
def fraction_multiple(series):
    return np.sum(series[series>1])/np.sum(series)

In [ ]:
k = 9
filterlength = 12

In [ ]:
count_series = counter_to_series(count_kmers_iterable(df['Sequence'], k, clean=True))
fraction_multiple(count_series)

In [ ]:
#filtered = df.groupby('TaxID_species').head(20)
filtered = df.groupby(['TaxID_species', 'Protein name']).head(1)

In [ ]:
#write_fasta(filtered, datadir+'human-viruses-uniref90-filtered.fasta',
#            seqcolumn='Sequence', descriptioncolumn='header')

In [ ]:
count_series_filtered = counter_to_series(count_kmers_iterable(filtered['Sequence'], k, clean=True))
fraction_multiple(count_series_filtered)

In [ ]:
count_series_filtered_unique = counter_to_series(
                       count_kmers_iterable(
                           filter_unique(filtered['Sequence'], k, filterlength),
                           k)
                       )
fraction_multiple(count_series_filtered_unique)

In [ ]:
'all %e, filtered %e, unique %e'%(np.sum(df['Sequence'].str.len()),
              np.sum(filtered['Sequence'].str.len()),
              np.sum(count_series_filtered_unique))

In [ ]:
count_series_filtered_unique.sort_values().tail(10)

In [ ]:
counts, bins, _ = plt.hist(count_series_filtered, log=True,
                           bins=np.arange(0, count_series_filtered.max()+1, 1+count_series_filtered.max()//1000));
plt.hist(count_series_filtered_unique, log=True, bins=bins);
plt.xscale('log')